In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torchvision
import torch.utils.data as tud
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
train_path = "dataset/Training"
test_path = "dataset/Testing"

In [ ]:
#transform on train data
imgSize = 32
transTrain = transforms.Compose([transforms.ToTensor(),transforms.Resize((imgSize,imgSize)),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
                                transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2)),
                                transforms.RandomRotation(20) ])
#transform on test data
transTest = transforms.Compose([transforms.ToTensor(),transforms.Resize((imgSize,imgSize)),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))])

In [ ]:
valPer = 0.1
batchSize = 30

trainImgs = datasets.ImageFolder(train_path, transform=transTrain)
testImgs = datasets.ImageFolder(test_path, transform=transTrain)


trainSize = len(trainImgs)
idxs = list(range(trainSize))
np.random.shuffle(idxs)
spl = int(np.floor(valPer * trainSize))          #split the dataset to train and validation sets
trainIdxs, valIdxs = idxs[spl:], idxs[:spl]

trainSamp = SubsetRandomSampler(trainIdxs)
valSamp = SubsetRandomSampler(valIdxs)


trainLoader = tud.DataLoader(trainImgs,batch_size=batchSize,sampler=trainSamp,num_workers=0)
valLoader = tud.DataLoader(trainImgs,batch_size=batchSize,sampler=valSamp,num_workers=0)
testLoader = tud.DataLoader(testImgs,batch_size=batchSize,num_workers=0,shuffle=True)



In [ ]:
#Defining the cnn network
class CNN(nn.Module):
  def __init__(self):
      super(CNN,self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
      self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
      self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
      self.fc1 = nn.Linear(64 * 4 * 4, 100)
      self.fc2 = nn.Linear(100, 4)
      self.maxPool = nn.MaxPool2d(kernel_size = 2, stride = 2)
      self.dropout = nn.Dropout(0.25)
      self.relu = nn.ReLU()

  def forward(self,x):
      out = self.maxPool(func.relu(self.conv1(x)))
      out = self.maxPool(func.relu(self.conv2(out)))
      out = self.maxPool(func.relu(self.conv3(out)))
      out = out.view(-1, 64 * 4 * 4)
      out = self.dropout(out)
      out = func.relu(self.fc1(out))
      out = self.dropout(out)
      out = self.fc2(out)
      return out

In [ ]:
#Training the model
import torch.optim as optim

cnnNet = CNN()
cnnNet.to(device)
criterion = nn.CrossEntropyLoss()   # Settin Loss function with criterion
lr = 0.01 #learning rate
optimizer = optim.SGD(cnnNet.parameters(), lr=lr, momentum = 0.9)

nepc = 20 # number of epochs


desiredLoss = 0.001
minLoss = np.Inf
correct = 0
for epc in range(1,nepc+1):
    trainLoss = 0
    cnnNet.train()
    #-----------Training
    for images,labels in trainLoader: 
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      print(len(images))
      pred = cnnNet(images)
      pred = pred.to(dtype=torch.float)
      # _, pred = torch.max(pred.data, 1)
      # correct += (pred == labels).sum().item()
      loss = criterion(pred, labels)
      loss.backward()
      optimizer.step()
      trainLoss += loss.item()
    trainLoss = trainLoss/len(trainLoader)
    print('Epoch no {} , Loss: {:.4f}'.format(epc, trainLoss))
   
    #-----------Validation
    cnnNet.eval()
    valLoss = 0
    valLoss2 = 0
    for images,labels in valLoader:
      images = images.to(device)
      labels = labels.to(device)
      print(len(labels))
      pred = cnnNet(images)
      # pred = pred.to(dtype=torch.float)
      # _, pred = torch.max(pred.data, 1)
      pred = pred.to(dtype=torch.float)
      loss = criterion(pred, labels)
      valLoss += loss.item()

    valLoss = valLoss/len(valLoader)
    if valLoss < minLoss:  #saving the model with minimum loss
      torch.save(cnnNet.state_dict(), 'cnnNet')
      minLoss = valLoss
      print('Validation loss: {:.4f}'.format(valLoss))
    if valLoss < desiredLoss:
      break


In [ ]:
#Evluating the model
cnnNet.load_state_dict(torch.load('cnnNet'))  #Loading the best model, with minimum error

testLoss = 0
correct = 0
cnnNet.eval()
total = 0
for images,labels in testLoader:
    images = images.to(device)
    labels = labels.to(device)
    pred = cnnNet(images)
    pred = pred.to(dtype=torch.float)
    loss = criterion(pred, labels)
    testLoss += loss.item()*images.size(0)
    total += labels.size(0)
    _, pred = torch.max(pred, 1)
    correct += (pred == labels).sum().item()
testLoss = testLoss/len(testLoader)

print('The final accuracy of the network on brain cancer images: {:.4f}'.format(correct/total*100))


The final accuracy of the network on brain cancer images: 74.1419
